In [7]:
import backtrader as bt
import backtrader.feeds as btfeeds

import pandas as pd
import yfinance as yf
from datetime import date, timedelta


interval = "1h"
interval_map = {
    "1m": timedelta(minutes=60*24*7),
    "1h": timedelta(hours=24*90),
    "1d": timedelta(days=365*1),   
}

ticker0 = "BZ=F"
ticker1 = "CL=F"

today = date.today()

# Backtrader needs datetime objects
end_date = today
start_date = today - interval_map[interval]

# Yahoo needs strings
end_date_str = end_date.strftime("%Y-%m-%d")
start_date_str = start_date

df0 = yf.download(ticker0, start=start_date_str, end=end_date_str, interval=interval)
df1 = yf.download(ticker1, start=start_date_str, end=end_date_str, interval=interval)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
# Shorten data while testing
df0 = df0.tail(10)
df1 = df1.tail(10)

data_path0 = "data/df1.csv"
data_path1 = "data/df2.csv"
df0.to_csv(data_path0)
df1.to_csv(data_path1)

In [10]:
class OUPairsTradingStrategy(bt.Strategy):

    def __init__(self):
        print(self.position)
        print(self.data)
        
    def next(self):
        print(f"Close 0 | {self.data0.close[0]} | {self.data1.close[0]} | Close 1")

In [11]:
cb = bt.Cerebro()

# Add the data streams to the engine
data0 = btfeeds.YahooFinanceCSVData(
    dataname=data_path0,
    fromdate=start_date,
    todate=end_date,
)
cb.adddata(data0)

data1 = btfeeds.YahooFinanceCSVData(
    dataname=data_path1,
    fromdate=start_date,
    todate=end_date,
)
cb.adddata(data1)

# Add the trading strategy to the engine
cb.addstrategy(OUPairsTradingStrategy)

# Set starting cash balance
cb.broker.setcash(100.0)

# Percentage commission - broker fees: 0.005 is 0.5%.
cb.broker.setcommission(0.005)

# Initial conditions
print(f"Starting Portfolio Value: {cb.broker.getvalue()}")

cb.run()

# Result
print(f"Final Portfolio Value: {cb.broker.getvalue()}")

Starting Portfolio Value: 100.0
--- Position Begin
- Size: 0
- Price: 0.0
- Price orig: 0.0
- Closed: 0
- Opened: 0
- Adjbase: None
--- Position End
Close 0 | 94.66 | 88.3 | Close 1
Close 0 | 93.62 | 87.26 | Close 1
Close 0 | 93.2 | 86.65 | Close 1
Close 0 | 93.35 | 86.78 | Close 1
Close 0 | 93.59 | 86.81 | Close 1
Close 0 | 92.91 | 86.14 | Close 1
Close 0 | 92.65 | 85.75 | Close 1
Close 0 | 92.44 | 85.62 | Close 1
Close 0 | 92.4 | 85.64 | Close 1
Close 0 | 92.57 | 85.81 | Close 1
Final Portfolio Value: 100.0
